# 📘 COVID-19 Global Data Report

In [ ]:
# 📌 Section 1: Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime

# Optional: Enable interactive widgets
from ipywidgets import interact, widgets
from IPython.display import display

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
# 📌 Section 2: Load Data
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
df = pd.read_csv(url, parse_dates=['date'])
df.head()

In [ ]:
# 📌 Section 3: Data Cleaning
columns = [
    'iso_code', 'continent', 'location', 'date',
    'total_cases', 'new_cases', 'total_deaths', 'new_deaths',
    'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
    'hosp_patients', 'icu_patients', 'population'
]
df = df[columns]
df.sort_values(['location', 'date'], inplace=True)
df.fillna(method='ffill', inplace=True)
df = df[df['continent'].notna()]

In [ ]:
# 📌 Section 4: User Input with Widgets (Optional)
countries = df['location'].unique().tolist()
min_date = df['date'].min()
max_date = df['date'].max()

country_selector = widgets.Dropdown(options=countries, value="United States", description='Country:')
date_range_slider = widgets.SelectionRangeSlider(
    options=pd.date_range(start=min_date, end=max_date, freq='D'),
    index=(0, len(pd.date_range(start=min_date, end=max_date, freq='D')) - 1),
    description='Date Range:',
    layout={'width': '90%'}
)
display(country_selector, date_range_slider)

In [ ]:
# 📌 Section 5: Filtered Dataset Function
def get_filtered_data(country, start_date, end_date):
    mask = (
        (df['location'] == country) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    )
    return df[mask]

In [ ]:
# 📌 Section 6: EDA & Visualizations
country = country_selector.value
start_date, end_date = date_range_slider.value

df_filtered = get_filtered_data(country, start_date, end_date)

# Total Cases
fig = px.line(df_filtered, x='date', y='total_cases', title=f'Total COVID-19 Cases in {country}')
fig.show()

# Death Rate
df_filtered['death_rate'] = df_filtered['total_deaths'] / df_filtered['total_cases']
fig = px.line(df_filtered, x='date', y='death_rate', title=f'Death Rate in {country}')
fig.show()

# Vaccination Rate
df_filtered['vaccinated_pct'] = df_filtered['people_vaccinated'] / df_filtered['population']
fig = px.line(df_filtered, x='date', y='vaccinated_pct', title=f'Vaccination Coverage in {country}')
fig.show()

# Hospitalization
if df_filtered['hosp_patients'].notna().sum() > 0:
    fig = px.line(df_filtered, x='date', y='hosp_patients', title=f'Hospitalized Patients in {country}')
    fig.show()

In [ ]:
# 📌 Section 7: Country Comparison
latest = df[df['date'] == df['date'].max()]
latest = latest[latest['total_cases'].notna()]
top10 = latest.sort_values('total_cases', ascending=False).head(10)
fig = px.bar(top10, x='location', y='total_cases', title='Top 10 Countries by Total Cases')
fig.show()

# 📌 Section 8: Summary & Insights
- What patterns are clear?
- Are cases rising or falling?
- How does vaccination affect the death rate?
- Add your observations and conclusions here.